In [1]:
!pip install pytreebank

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp37-none-any.whl size=37070 sha256=4ec3bfafd96c784282cd267c8955a1685a814a89aced9ae48f2ea46dda3da367
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank


In [2]:
import pytreebank
import pandas as pd

In [3]:
dataset = pytreebank.load_sst()

In [4]:
dataset['test'][456].to_labeled_lines()[0]

(3,
 'Jason X has cheesy effects and a hoary plot , but its macabre , self-deprecating sense of humor makes up for a lot .')

In [5]:
len(dataset['train']), len(dataset['test']), len(dataset['dev'])

(8544, 2210, 1101)

In [6]:
[dataset.keys()]

[dict_keys(['train', 'test', 'dev'])]

In [7]:
train_df = pd.DataFrame({'sentence' : [dataset['train'][i].to_labeled_lines()[0][1] for i in range(len(dataset['train']))], 'labels' : [dataset['train'][i].to_labeled_lines()[0][0] for i in range(len(dataset['train']))]})
test_df = pd.DataFrame({'sentence' : [dataset['test'][i].to_labeled_lines()[0][1] for i in range(len(dataset['test']))], 'labels' : [dataset['test'][i].to_labeled_lines()[0][0] for i in range(len(dataset['test']))]})
val_df = pd.DataFrame({'sentence' : [dataset['dev'][i].to_labeled_lines()[0][1] for i in range(len(dataset['dev']))], 'labels' : [dataset['dev'][i].to_labeled_lines()[0][0] for i in range(len(dataset['dev']))]})
df = pd.concat([train_df, test_df, val_df]).reset_index()
df.drop('index', axis=1, inplace=True)
#df.set_index('labels', inplace=True)


In [8]:
df.tail()

,sentence,labels
11850,it seems to me the film is about the art of ri...,1
11851,It 's just disappointingly superficial -- a mo...,1
11852,The title not only describes its main characte...,1
11853,Sometimes it feels as if it might have been ma...,2
11854,Schaeffer has to find some hook on which to ha...,1


In [9]:
df.loc[9000]

sentence    Jason X has cheesy effects and a hoary plot , ...
labels                                                      3
Name: 9000, dtype: object

In [10]:
df.shape

(11855, 2)

In [11]:
df.labels.value_counts()

1    3140
3    3111
2    2242
4    1852
0    1510
Name: labels, dtype: int64

In [42]:
df_train = df[:int(0.7*len(df))]
df_val = df[len(df_train):]

In [25]:
df_train.head(), df_val.head()

(                                            sentence  labels
 0  The Rock is destined to be the 21st Century 's...       3
 1  The gorgeously elaborate continuation of `` Th...       4
 2  Singer/composer Bryan Adams contributes a slew...       3
 3  You 'd think by now America would have had eno...       2
 4               Yet the act is still charming here .       3,
                                                sentence  labels
 8298  A prison comedy that never really busts out of...       1
 8299  The movie is obviously a labour of love so How...       2
 8300            This is n't a movie ; it 's a symptom .       1
 8301  What we have is a character faced with the pos...       0
 8302                 If only it were , well , funnier .       1)

In [26]:
len(df_train), len(df_val)

(8298, 3557)

In [27]:
! pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 1.0MB 10.0MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=0bcf801cbf3ee54f283a37dec8f958c858bd0e5120cf0a370ccd2ec8465f79f1
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [35]:
import random
import googletrans

translator = googletrans.Translator()
indexes = random.choices(list(range(len(df_train))), k = len(df_train)//16)                         
sentence = [df_train['sentence'][i] for i in indexes]

available_langs = list(googletrans.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, dest=trans_lang) 
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
en_text = [t.text for t in translations_en_random]
print(en_text)

Translating to khmer
["There 's something poignant about an artist of 90-plus years taking the effort to share his impressions of life and loss and time and art with us .", 'The film proves unrelentingly grim -- and equally engrossing .', "Chris Columbus ' sequel is faster , livelier and a good deal funnier than his original .", "It 's enough to watch Huppert scheming , with her small , intelligent eyes as steady as any noir villain , and to enjoy the perfectly pitched web of tension that Chabrol spins .", 'The movie would seem less of a trifle if Ms. Sugarman followed through on her defiance of the saccharine .', 'Happy Times maintains an appealing veneer without becoming too cute about it .', 'This is a movie that refreshes the mind and spirit along with the body , so original is its content , look , and style .', "Because Eight Legged Freaks is partly an homage to Them , Tarantula and other low - budget B-movie thrillers of the 1950s and '60s , the movie is a silly ( but not sophomo

In [36]:
backtranslated_dict={'sentence':[], 'labels':[]}
backtranslated_dict['sentence'] = [sentence for sentence in en_text]
backtranslated_dict['labels'] = [df_train['labels'][i] for i in indexes]

In [41]:
indexes[-1]

1905

In [42]:
df_train.loc[1905]

sentence    Huppert and Girardot give performances of exce...
labels                                                      4
Name: 1905, dtype: object

In [39]:
backtranslated_dict

{'labels': [3,
  3,
  3,
  3,
  1,
  3,
  4,
  2,
  3,
  3,
  1,
  1,
  2,
  2,
  1,
  2,
  1,
  4,
  0,
  0,
  2,
  4,
  2,
  4,
  2,
  2,
  3,
  2,
  3,
  2,
  2,
  0,
  0,
  3,
  0,
  0,
  3,
  3,
  4,
  3,
  4,
  3,
  3,
  1,
  1,
  3,
  2,
  3,
  2,
  0,
  0,
  4,
  3,
  4,
  3,
  1,
  0,
  2,
  4,
  2,
  4,
  1,
  0,
  1,
  1,
  1,
  3,
  1,
  1,
  0,
  3,
  1,
  1,
  3,
  0,
  4,
  1,
  4,
  4,
  0,
  1,
  1,
  0,
  0,
  2,
  3,
  3,
  1,
  3,
  3,
  4,
  1,
  1,
  3,
  1,
  3,
  2,
  1,
  0,
  2,
  2,
  1,
  4,
  1,
  3,
  1,
  2,
  4,
  2,
  3,
  0,
  1,
  1,
  2,
  1,
  4,
  0,
  3,
  0,
  1,
  0,
  1,
  4,
  1,
  1,
  2,
  3,
  0,
  4,
  3,
  2,
  0,
  1,
  2,
  4,
  1,
  4,
  4,
  1,
  2,
  3,
  3,
  1,
  1,
  3,
  4,
  0,
  1,
  1,
  1,
  1,
  4,
  3,
  1,
  0,
  1,
  0,
  0,
  4,
  3,
  1,
  1,
  1,
  4,
  4,
  3,
  4,
  3,
  4,
  0,
  2,
  1,
  4,
  3,
  3,
  3,
  0,
  3,
  2,
  3,
  4,
  2,
  0,
  3,
  3,
  1,
  2,
  3,
  2,
  3,
  1,
  3,
  4,
  3,
  4,
  1,
  0,
  2,


In [43]:
df_aug = pd.DataFrame(backtranslated_dict)
df_train_aug = pd.concat([df_train, df_aug]).reset_index()
df_train_aug.drop('index', axis=1, inplace=True)

In [47]:
len(df_train_aug)

8816

In [48]:
df_train_aug['labels'][0]

3

In [50]:
with open('augmented_dataset.txt','w') as f:
  for i in range(len(df_train_aug)):
    f.write(f"{df_train_aug['labels'][i]}\t{df_train_aug['sentence'][i]}\n")

In [67]:
#for i in backtranslated_dict:
 # df_train['sentence'][i] = backtranslated_dict[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [52]:
!git clone "https://github.com/jasonwei20/eda_nlp"

Cloning into 'eda_nlp'...
remote: Enumerating objects: 396, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 396 (delta 6), reused 8 (delta 4), pack-reused 379
Receiving objects: 100% (396/396), 20.42 MiB | 21.35 MiB/s, done.
Resolving deltas: 100% (189/189), done.


In [53]:
!pip install -U nltk

     |████████████████████████████████| 1.5MB 6.5MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [54]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [58]:
%cd 'eda_nlp'

/content/eda_nlp


In [59]:
!python code/augment.py --input='/content/eda_nlp/data/augmented_dataset.txt'

generated augmented sentences with eda for /content/eda_nlp/data/augmented_dataset.txt to /content/eda_nlp/data/eda_augmented_dataset.txt with num_aug=9


# Train Prep

In [13]:
import random
import torch, torchtext
from torchtext import data

In [14]:
# Manual Seed
SEED = 43
torch.manual_seed(SEED)
CUDA_LAUNCH_BLOCKING=1

In [31]:
df_train = pd.read_csv('eda_augmented_dataset.txt', sep='\t')
df_val = df_val

In [32]:
df_train.head()

,labels,sentence
0,3,he rock is destined to be the st century s seg...
1,3,the rock is destined to be the st century s ne...
2,3,the rock is destined to be the st c s new cona...
3,3,the rock is destined to be the st century s no...
4,3,to rock is destined to be the st century s new...


In [43]:
df_val = df_val.reset_index()
df_val.drop('index', axis=1, inplace=True)

In [16]:
Sentence = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [33]:
fields = [('sentence', Sentence), ('label', Label)]

In [34]:
example_train = [torchtext.legacy.data.Example.fromlist([df_train.sentence[i],df_train.labels[i]], fields) for i in range(df_train.shape[0])] 




KeyError: ignored

In [44]:
example_val = [torchtext.legacy.data.Example.fromlist([df_val.sentence[i],df_val.labels[i]], fields) for i in range(df_val.shape[0])] 

In [45]:
train = torchtext.legacy.data.Dataset(example_train, fields)
valid = torchtext.legacy.data.Dataset(example_val, fields)

In [ ]:
#(train, valid) = dataset.split(split_ratio=[70, 30], random_state = random.seed(SEED))


In [46]:
len(train), len(valid)

(88160, 3557)

In [ ]:
df_train.head()

In [47]:
Sentence.build_vocab(train)
Label.build_vocab(train)

In [48]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  21348
Size of label vocab :  5
Top 10 words appreared repeatedly : [('the', 76582), ('a', 56433), ('and', 46729), ('of', 46418), ('to', 32101), ('s', 26994), ('is', 26571), ('it', 24998), ('that', 20477), ('in', 20454)]
Labels :  defaultdict(None, {3: 0, 1: 1, 2: 2, 4: 3, 0: 4})


In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [50]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((train, valid), batch_size = 16, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

In [51]:
next(iter(train_iterator))
#len(train.examples[11].tweet)


[torchtext.legacy.data.batch.Batch of size 16]
	[.sentence]:('[torch.cuda.LongTensor of size 16x17 (GPU 0)]', '[torch.cuda.LongTensor of size 16 (GPU 0)]')
	[.label]:[torch.cuda.LongTensor of size 16 (GPU 0)]

In [52]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Sentence.vocab.stoi, tokens)

In [53]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [54]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [55]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(21348, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 6,646,505 trainable parameters


In [56]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [57]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        # resets the gradients after every batch
        optimizer.zero_grad() 
        
        
        # retrieve text and no. of words
        sentence, sentence_lengths = batch.sentence  
        
        # convert to 1D tensor
        predictions = model(sentence, sentence_lengths).squeeze()  
        #print(predictions, batch.label)
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [58]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            sentence, sentence_lengths = batch.sentence
            
            # convert to 1d tensor
            predictions = model(sentence, sentence_lengths).squeeze() 
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [59]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.423 | Train Acc: 48.30%
	 Val. Loss: 1.564 |  Val. Acc: 31.09% 

	Train Loss: 1.175 | Train Acc: 74.46%
	 Val. Loss: 1.569 |  Val. Acc: 30.92% 

	Train Loss: 1.069 | Train Acc: 84.86%
	 Val. Loss: 1.585 |  Val. Acc: 28.59% 

	Train Loss: 0.995 | Train Acc: 91.89%
	 Val. Loss: 1.607 |  Val. Acc: 26.83% 

	Train Loss: 0.965 | Train Acc: 94.49%
	 Val. Loss: 1.601 |  Val. Acc: 27.92% 

	Train Loss: 0.951 | Train Acc: 95.64%
	 Val. Loss: 1.589 |  Val. Acc: 29.27% 

	Train Loss: 0.943 | Train Acc: 96.40%
	 Val. Loss: 1.608 |  Val. Acc: 27.30% 

	Train Loss: 0.937 | Train Acc: 96.89%
	 Val. Loss: 1.610 |  Val. Acc: 27.05% 

	Train Loss: 0.933 | Train Acc: 97.32%
	 Val. Loss: 1.589 |  Val. Acc: 29.61% 

	Train Loss: 0.930 | Train Acc: 97.55%
	 Val. Loss: 1.613 |  Val. Acc: 27.30% 



In [60]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_sentence(sentence):
    
    categories = {0: "Very Negative", 1:"Negative", 2:"Neutral", 3:"Positive", 4:"Very Positive"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [61]:
classify_sentence("Jason X has cheesy effects and a hoary plot , but its macabre , self-deprecating sense of humor makes up for a lot .")

'Negative'